In [49]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from datetime import datetime
from typing import Union
import logging
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


def extract_test_data(entity):
    
    query = f"""
        SELECT des.entity_id
            ,des.test_id
            ,des.test_start_date
            ,des.test_end_date
            ,des.variation_group
            ,des.variation_share
        FROM `fulfillment-dwh-production.cl.dps_experiment_setups` des
        WHERE des.test_start_date > '2024-01-01'
        AND des.entity_id = '{entity}'  -- Ensure entity is properly injected
        AND test_start_date is not null
        --AND test_id > 5
        GROUP BY 1,2,3,4,5,6
        ORDER BY des.entity_id, des.test_start_date
    """
    
    return query


project_id = "logistics-customer-staging"
logging.info(f"Initializing BigQuery client for project: {project_id}")

try:
    # Initialize BigQuery client
    client = bigquery.Client(project=project_id)
except Exception as e:
    logging.error(f"Failed to initialize BigQuery client: {e}")
    sys.exit(1)

entity_id = 'GV_RS'

data_test = client.query(extract_test_data(entity_id)).to_dataframe()

/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [51]:
data_test

,entity_id,test_id,test_start_date,test_end_date,variation_group,variation_share
0,GV_RS,1,2024-11-14 12:12:50.257612+00:00,2024-11-18 20:37:27.870038+00:00,Control,12.00
1,GV_RS,1,2024-11-14 12:12:50.257612+00:00,2024-11-18 20:37:27.870038+00:00,Variation3,12.00
2,GV_RS,1,2024-11-14 12:12:50.257612+00:00,2024-11-18 20:37:27.870038+00:00,Variation4,12.00
3,GV_RS,1,2024-11-14 12:12:50.257612+00:00,2024-11-18 20:37:27.870038+00:00,Variation1,12.00
4,GV_RS,1,2024-11-14 12:12:50.257612+00:00,2024-11-18 20:37:27.870038+00:00,Variation2,12.00
5,GV_RS,2,2024-11-18 20:37:43.831257+00:00,2025-02-06 09:22:38.233211+00:00,Control,25.00
6,GV_RS,2,2024-11-18 20:37:43.831257+00:00,2025-02-06 09:22:38.233211+00:00,Variation2,25.00
7,GV_RS,2,2024-11-18 20:37:43.831257+00:00,2025-02-06 09:22:38.233211+00:00,Variation3,25.00
8,GV_RS,2,2024-11-18 20:37:43.831257+00:00,2025-02-06 09:22:38.233211+00:00,Variation1,25.00
9,GV_RS,3,2025-01-14 13:44:18.709964+00:00,2025-01-23 17:20:08.165484+00:00,Control,50.00


In [50]:
# Ensure 'test_start_date' and 'test_end_date' are properly converted to datetime
data_test["test_start_date"] = pd.to_datetime(data_test["test_start_date"], errors="coerce")
data_test["test_end_date"] = pd.to_datetime(data_test["test_end_date"], errors="coerce")

# Replace missing 'test_end_date' values with the current timestamp
current_date = pd.Timestamp.now()
data_test.loc[data_test["test_end_date"].isna(), "test_end_date"] = current_date

# Recalculate duration after filling missing values
data_test["duration"] = (data_test["test_end_date"] - data_test["test_start_date"]).dt.days

# Sort by start date for better visualization
data_test = data_test.sort_values(by="test_start_date")

# Adjust figure size dynamically based on number of tests
fig_height = max(6, len(data_test) * 0.3)  # Ensures proper spacing
fig, ax = plt.subplots(figsize=(12, fig_height))

# Create a horizontal bar for each test
for i, row in data_test.iterrows():
    start_date = row["test_start_date"]
    duration = row["duration"]
    test_id = row["test_id"]

    # Ensure a minimum bar length for visibility
    min_duration = max(duration, 1)  

    # Plot the horizontal bar
    ax.barh(test_id, min_duration, left=start_date, color="skyblue", edgecolor="black")

    # Add Test ID label inside the bar
    ax.text(start_date + pd.Timedelta(days=min_duration / 2), test_id, str(test_id),
            va='center', ha='center', fontsize=10, color="black", fontweight='bold')

# Format X-axis as months
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.xticks(rotation=45)

# Labels and title
ax.set_xlabel("Months")
ax.set_ylabel("Test ID")
ax.set_title("Experiment Timetable")

plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.tight_layout()  # Adjust layout to fit all bars properly
plt.show()


/var/folders/7c/hjrbzbpn61jgnsn14y5f3l7c0000gq/T/ipykernel_84103/605382987.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2025-02-28 23:25:24.071388' has dtype incompatible with datetime64[us, UTC], please explicitly cast to a compatible dtype first.
  data_test.loc[data_test["test_end_date"].isna(), "test_end_date"] = current_date


TypeError: cannot subtract DatetimeArray from ndarray